## Deliverable 2. Create a Customer Travel Destinations Map.

In [40]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [41]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vaini,-21.2000,-175.2000,81.57,69,100,13.44,TO,2022-01-31 02:39:24
1,1,Upernavik,72.7868,-56.1549,-7.04,97,100,3.85,GL,2022-01-31 02:39:24
2,2,Busselton,-33.6500,115.3333,74.08,37,99,16.28,AU,2022-01-31 02:39:24
3,3,Kirakira,-10.4544,161.9205,84.79,74,63,7.18,SB,2022-01-31 02:39:25
4,4,Rikitea,-23.1203,-134.9692,77.41,64,100,5.79,PF,2022-01-31 02:39:25


In [42]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = (float(input("What is the minimum temperature you would like for your trip? ")))
max_temp = (float(input("What is the maximum temperature you would like for your trip? ")))
max_temp

What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 50


50.0

In [43]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vaini,-21.2000,-175.2000,81.57,69,100,13.44,TO,2022-01-31 02:39:24
1,1,Upernavik,72.7868,-56.1549,-7.04,97,100,3.85,GL,2022-01-31 02:39:24
2,2,Busselton,-33.6500,115.3333,74.08,37,99,16.28,AU,2022-01-31 02:39:24
3,3,Kirakira,-10.4544,161.9205,84.79,74,63,7.18,SB,2022-01-31 02:39:25
4,4,Rikitea,-23.1203,-134.9692,77.41,64,100,5.79,PF,2022-01-31 02:39:25


In [44]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       113
City          113
Lat           113
Lng           113
Max Temp      113
Humidity      113
Cloudiness    113
Wind Speed    113
Country       113
Date          113
dtype: int64

In [45]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_cities_clean_df = preferred_cities_df.dropna()
new_cities_clean_df.shape

(113, 10)

In [47]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_cities_clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()


,City,Country,Max Temp,Lat,Lng,Hotel Name
19,Muros,ES,49.23,42.7762,-9.0603,
28,Gafanha Da Encarnacao,PT,45.28,40.6181,-8.7330,
35,Golynki,RU,27.73,54.8696,31.3861,
36,Severo-Kurilsk,RU,28.45,50.6789,156.1250,
39,Punta Arenas,CL,48.31,-53.1500,-70.9167,


In [48]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
        print(f"hotel : {hotels['results'][0]['name']}")
    except (IndexError):
        print("Hotel not found... skipping.")

hotel : Apartment Av. Castelao
hotel : Hotel Barra
Hotel not found... skipping.
hotel : Kak Doma
hotel : Hotel Hain
hotel : Hotelʹ
hotel : Casa Maquia
hotel : Abbott's Bed & Breakfast
hotel : Hotel Arbe
hotel : Jinjiang Inn Style Qingzhou Walmart
Hotel not found... skipping.
Hotel not found... skipping.
hotel : The Hotel Petropavlovsky
hotel : ARTIEM Capri
hotel : mohamed ben khlil omri
hotel : Guangyuan Poetry Hotel
hotel : Baymont by Wyndham Glendive
hotel : Леонид рахманов
hotel : Tourist House - Sablino
hotel : Pension Feldgärtenhof
hotel : Yazioren Koyu Muhtarligi
hotel : Albatross Hotel
hotel : Holiday Inn Express & Suites Rolla - Univ of Missouri S&T, an IHG Hotel
hotel : Hotel Sierra de Las Villas
hotel : Boutique Waterfront Lodge
hotel : Hotel Olafsvik
hotel : Quality Inn & Suites Banquet Center
hotel : Northern Light Inn & Max's Restaurant
hotel : Hotel Arena Gruesa
hotel : Wajiima Onsen Yashio
hotel : Marystown Hotel & Convention Centre
hotel : Grand Hotel Cirta
hotel : Ston

In [49]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City          113
Country       113
Max Temp      113
Lat           113
Lng           113
Hotel Name    113
dtype: int64

In [66]:
import numpy as np
hotel_df.replace("", np.nan, inplace = True)
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
19,Muros,ES,49.23,42.7762,-9.0603,Apartment Av. Castelao
28,Gafanha Da Encarnacao,PT,45.28,40.6181,-8.7330,Hotel Barra
36,Severo-Kurilsk,RU,28.45,50.6789,156.1250,Kak Doma
39,Punta Arenas,CL,48.31,-53.1500,-70.9167,Hotel Hain
40,Lyubashivka,UA,34.86,47.8372,30.2598,Hotelʹ
...,...,...,...,...,...,...
669,Bani Walid,LY,46.40,31.7566,13.9942,منزل مفتاح محمد الزلبي
671,Tawang,IN,36.61,27.5833,91.8667,Hotel Mount View
675,Florence,IT,49.53,43.7667,11.2500,Italiana Hotels Florence
676,Jaca,ES,42.01,42.5690,-0.5499,Hotel Real Jaca & Spa


In [51]:
hotel_clean_df.count()

City          108
Country       108
Max Temp      108
Lat           108
Lng           108
Hotel Name    108
dtype: int64

In [73]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv

hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [71]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [74]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_clean_df[["Lat", "Lng"]]
max_temp = hotel_clean_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))